In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import os
import random
import pandas as pd 
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import load_coco_json
from sklearn.cluster import AgglomerativeClustering

In [2]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS =r'/opt/ice-ai-training-data/page_segment/all_data_v2/output/model_final.pth'

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
predictor = DefaultPredictor(cfg)

In [3]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
register_coco_instances("trainpsv2_23062020", {}, r"/opt/ice-ai-training-data/page_segment/all_data_v2/v2.json",
                        r'/opt/ice-ai-training-data/page_segment/all_data_v2/img')
"trainpsv2_23062020" in DatasetCatalog.list()

True

In [4]:
metadata = MetadataCatalog.get("trainpsv2_23062020")
dataset_dicts = DatasetCatalog.get("trainpsv2_23062020")
print(metadata)

WARNING [07/02 01:54:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/02 01:54:39 d2.data.datasets.coco]: Loaded 142 images in COCO format from /opt/ice-ai-training-data/page_segment/all_data_v2/v2.json
Metadata(evaluator_type='coco', image_root='/opt/ice-ai-training-data/page_segment/all_data_v2/img', json_file='/opt/ice-ai-training-data/page_segment/all_data_v2/v2.json', name='trainpsv2_23062020', thing_classes=['handw', 'printw', 'signature'], thing_dataset_id_to_contiguous_id={0: 0, 1: 1, 2: 2})


In [5]:
path = r"/opt/ice-ai-training-data/page_segment/all_data/test"
print('the number of image for prediction is : ' + str(len(os.listdir(path))) )
#os.mkdir("pred_image")


the number of image for prediction is : 10


In [122]:
os.listdir(path)[4]
#234052pdf_6_bilateral_handy

'234052pdf_6_bilateral.png'

In [123]:
#im = cv2.imread(os.path.join(path ,'234052pdf_6_bilateral_handy.png'))
im = cv2.imread(os.path.join(path , os.listdir(path)[4]))
outputs= predictor(im)
v = Visualizer(im[:, :, ::-1], metadata = metadata, scale=0.8)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite(r"/opt/ice-ai-training-data/page_segment/all_data_v2/pred_data/image" + str(10)+".png" , v.get_image()[:, :, ::-1])

True

In [124]:
pred_class = outputs["instances"].pred_classes.cpu().numpy()
pred_annot= outputs["instances"].pred_boxes.tensor.cpu().numpy()
full_pred  = np.concatenate((pred_annot,pred_class.reshape(-1,1)), axis = 1)
# print(type(full_pred))

In [125]:
## segregte the Hand and Print
##classes=['handw', 'printw', 'signature'], thing_dataset_id_to_contiguous_id={0: 0, 1: 1, 2: 2})
hand_indx = [ i for i in range(len(pred_class)) if pred_class[i] == 0 ]
print_indx = [ i for i in range(len(pred_class)) if pred_class[i] == 1 ]
####
### take out the annotations for handwritting
hannot_df = pd.DataFrame(data = full_pred[hand_indx].astype('int'), columns=['x1','y1', 'x2', 'y2','class'])
pannot_df = pd.DataFrame(data = full_pred[print_indx].astype('int'), columns=['x1','y1', 'x2', 'y2','class'])
full_annot_df = pd.DataFrame(data = full_pred.astype('int') ,columns=['x1','y1', 'x2', 'y2','class'])
print(len(hannot_df), len(pannot_df))

7 7


In [126]:
full_annot_df[full_annot_df['class']==1]

,x1,y1,x2,y2,class
0,126,865,590,934,1
1,106,463,605,524,1
2,102,762,597,815,1
4,117,238,541,277,1
5,96,330,584,464,1
7,101,633,563,759,1
9,105,286,514,322,1


In [64]:
## checking tthe annotation
# img_path =  r'/opt/ice-ai-training-data/page_segment/all_data/test/178192pdf_11_bilateral.png'
# img = cv2.imread(img_path)
# cv2.rectangle(img, (92,1509),(639,1547), (155,0,255), 2)
# #cv2.rectangle(img, (hannot_array[5][0],hannot_array[5][1]),(hannot_array[5][2],hannot_array[5][3]), (255,140,0), 2)
# cv2.imwrite(r'/opt/ice-ai-training-data/page_segment/all_data/test/testbox.png', img)

In [127]:
## sort based on xmin the handwritting boxes and 
# hannot_df.sort_values(['y1','x1'], inplace = True)
# hannot_df.head()

In [128]:
cluster = AgglomerativeClustering(n_clusters=len(hannot_df),
                        affinity='euclidean', \
                        memory=None, \
                        connectivity=None, \
                        compute_full_tree='auto', \
                        linkage='single', \
                        distance_threshold=None)

In [129]:
cluster

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='single', memory=None, n_clusters=7)

In [15]:
#data_feature = full_annot_df['y1'].to_numpy().reshape(-1,1)

In [133]:
data_feature = full_annot_df['y1'].to_numpy().reshape(-1,1)

In [134]:
cluster.fit(data_feature)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='single', memory=None, n_clusters=7)

In [135]:
cluster

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='single', memory=None, n_clusters=7)

In [136]:
## checking tthe annotation
img_path =  r'/opt/ice-ai-training-data/page_segment/all_data/test/234052pdf_6_bilateral.png'
img = cv2.imread(img_path)
img_newbox = img.copy()
for i in range(len(hannot_df)):
    class_label = list(np.where(cluster.labels_ == i)[0])
    
    colour = np.random.randint(high = 255, low = 0, size = 1)
    for item in full_annot_df.iloc[class_label].iterrows():
        cv2.rectangle(img, (item[1]['x1'],item[1]['y1']),(item[1]['x2'],item[1]['y2']), (155,0,255), 2)
        cv2.putText(img,'class'+str(i), (item[1]['x1']+10,item[1]['y1']+10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,100), 2)
        #cv2.rectangle(img, (hannot_array[5][0],hannot_array[5][1]),(hannot_array[5][2],hannot_array[5][3]), (255,140,0), 2)
cv2.imwrite(r'/opt/ice-ai-training-data/page_segment/all_data_v2/mapping/234052pdf_6_bilateral_mappingy.png', img)


True

In [139]:
df_mappinghp = pd.DataFrame( columns= ['px1','py1','px2','py2','hx1','hy1','hx2','hy2'])

In [140]:
img_newbox = img.copy()
##### hand --> 0 print ---> 1 
for i in range(len(pannot_df)):

    class_label = list(np.where(cluster.labels_ == i)[0])
    #print(class_label)
    df_cluster = full_annot_df.iloc[class_label]
    df_handcluster = df_cluster[df_cluster['class'] == 0]
    df_printcluster = df_cluster[df_cluster['class']==1]
    #print('handcluster\n',df_handcluster,'\n\t\t####\nprintcluster\n', df_printcluster)
    ####
    if len(df_printcluster)==1 and len(df_handcluster) == 1:
        ##only one key value pair
        color_array = np.random.randint(low = 50 , high = 255, size = 3)
        color = (color_array.item(0),color_array.item(1),color_array.item(2))
        cv2.arrowedLine(img_newbox,(df_printcluster['x2'],df_printcluster['y2']-20),\
                 (df_handcluster['x1'],df_handcluster['y2']-20),color,2)
        df_mappinghp = df_mappinghp.append({'px1':df_printcluster['x1'].values[0] ,'py1':df_printcluster['y1'].values[0] ,'px2':df_printcluster['x2'].values[0] ,\
                                            'py2':df_printcluster['y2'].values[0] ,'hx1':df_handcluster['x1'].values[0] ,'hy1':df_handcluster['y1'].values[0] ,\
                                            'hx2':df_handcluster['x2'].values[0] ,'hy2':df_handcluster['y2'].values[0] }, ignore_index = True)
        

    else:
        # #--> make a occupied list for handwritting
        # list_handw= ['unassigned']*len(df_handcluster)
        # ####
        #--> sort the df_print y then x ( experimental)
        df_printcluster.sort_values(['y1','x1'], inplace = True)
        ###
        #print("\t\t###\n")
        #--> iterate over the print box 
        for print_row in df_printcluster.iterrows():
            py1 = print_row[1]['y1']
            py2 = print_row[1]['y2']
            px1 = print_row[1]['x1']
            px2 = print_row[1]['x2']
            ####
            list_priw = set(np.arange(py1,py2 ))
            #print(f'print row -->\n{print_row[1]}')
            py_mid =  (py1+py2)//2
            (py1,py2,py_mid)
            ####
            for hand_row in df_handcluster.iterrows():
                #print(f'hand row -->\n {hand_row}')
                list_handw = set(np.arange(hand_row[1]['y1'],hand_row[1]['y2']) )
                set_common_point = list_priw.intersection(list_handw)
                overlap_percent_print = len(set_common_point) / len(list_priw)
                overlap_percent_hand = len(set_common_point) / len(list_handw)
                if (overlap_percent_print >= 0.6 or overlap_percent_hand >= 0.6) and hand_row[1]['x1']>print_row[1]['x1'] :
                    #print(f'pixel match')
                    if abs(hand_row[1]['x1'] - px2)<img_newbox.shape[1]//3:
                        #print(f'box match')
                        color_array = np.random.randint(low = 100 , high = 255, size = 3)
                        color = (color_array.item(0),color_array.item(1),color_array.item(2))
                        cv2.arrowedLine(img_newbox,(print_row[1]['x2'],print_row[1]['y2']-20),\
                             (hand_row[1]['x1'],hand_row[1]['y2']-20),color,2)
                        df_mappinghp= df_mappinghp.append({'px1':print_row[1]['x1'] ,'py1':print_row[1]['y1'] ,\
                                             'px2':print_row[1]['x2'] ,'py2':print_row[1]['y2'] ,\
                                             'hx1':hand_row[1]['x1'] ,'hy1':hand_row[1]['y1'] ,'hx2':hand_row[1]['x2'] ,\
                                             'hy2':hand_row[1]['y2'] }, ignore_index = True)
                    
    cv2.imwrite(r'/opt/ice-ai-training-data/page_segment/all_data_v2/mapping/234052pdf_6_bilateral_lines.png',img_newbox)



    
    
    
    
    
    
    
    
    
    
    
    
    



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [141]:
df_mappinghp

,px1,py1,px2,py2,hx1,hy1,hx2,hy2
0,105,286,514,322,770,291,904,339
1,96,330,584,464,641,341,1488,454
2,126,865,590,934,641,866,783,932
3,102,762,597,815,650,766,775,830
4,106,463,605,524,676,473,1393,559
5,101,633,563,759,655,633,856,718


In [142]:
df__11 = df_mappinghp.copy()
df__11

,px1,py1,px2,py2,hx1,hy1,hx2,hy2
0,105,286,514,322,770,291,904,339
1,96,330,584,464,641,341,1488,454
2,126,865,590,934,641,866,783,932
3,102,762,597,815,650,766,775,830
4,106,463,605,524,676,473,1393,559
5,101,633,563,759,655,633,856,718


In [143]:
kk = df_mappinghp.groupby(['hx1','hx2','hy1','hy2'])

In [144]:
kk.groups

{(641, 783, 866, 932): Int64Index([2], dtype='int64'),
 (641, 1488, 341, 454): Int64Index([1], dtype='int64'),
 (650, 775, 766, 830): Int64Index([3], dtype='int64'),
 (655, 856, 633, 718): Int64Index([5], dtype='int64'),
 (676, 1393, 473, 559): Int64Index([4], dtype='int64'),
 (770, 904, 291, 339): Int64Index([0], dtype='int64')}

In [145]:
for group_name, df_group in kk:
    print(df_group, len(df_group))

   px1  py1  px2  py2  hx1  hy1  hx2  hy2
2  126  865  590  934  641  866  783  932 1
  px1  py1  px2  py2  hx1  hy1   hx2  hy2
1  96  330  584  464  641  341  1488  454 1
   px1  py1  px2  py2  hx1  hy1  hx2  hy2
3  102  762  597  815  650  766  775  830 1
   px1  py1  px2  py2  hx1  hy1  hx2  hy2
5  101  633  563  759  655  633  856  718 1
   px1  py1  px2  py2  hx1  hy1   hx2  hy2
4  106  463  605  524  676  473  1393  559 1
   px1  py1  px2  py2  hx1  hy1  hx2  hy2
0  105  286  514  322  770  291  904  339 1


In [ ]:
for group_name, df_group in kk:
    for item in df_group.iterrows():
        print(item[1]['px1'])
        

In [118]:
for group_name, df_group in kk:
    #print(group_name)
    print(type(df_group))
    print(type(min(df_group.index.values)))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.int64'>


In [147]:
df_mappinghp.to_csv('line.csv', index= False )

In [ ]:
class_label = list(np.where(cluster.labels_ == 9)[0])
#print(class_label)
df_cluster = full_annot_df.iloc[class_label]
df_handcluster = df_cluster[df_cluster['class'] == 0]
df_printcluster = df_cluster[df_cluster['class']==1]
print('handcluster\n',df_handcluster,'\n\t\t####\nprintcluster\n', df_printcluster)
df_printcluster.sort_values(['y1','x1'], inplace = True)


for print_row in df_printcluster.iterrows():
        py1 = print_row[1]['y1']
        py2 = print_row[1]['y2']
        px1 = print_row[1]['x1']
        px2 = print_row[1]['x2']
        ####
        list_priw = set(np.arange(py1,py2 ))
        #print(f'print row -->\n{print_row[1]}')
        py_mid =  (py1+py2)//2
        (py1,py2,py_mid)
        ####
        for hand_row in df_handcluster.iterrows():
            #print(f'hand row -->\n {hand_row}')
            list_handw = set(np.arange(hand_row[1]['y1'],hand_row[1]['y2']) )
            set_common_point = list_priw.intersection(list_handw)
            overlap_percent = len(set_common_point) / len(list_priw)
            if overlap_percent >= 0.6 and hand_row[1]['x1']>print_row[1]['x1'] :
                #print(f'pixel match')
                if abs(hand_row[1]['x1'] - px2)<img_newbox.shape[1]//3:
                    #print(f'box match')
                    color_array = np.random.randint(low = 100 , high = 255, size = 3)
                    color = (color_array.item(0),color_array.item(1),color_array.item(2))
                    cv2.arrowedLine(img_newbox,(print_row[1]['x2'],print_row[1]['y2']-20),\
                 (hand_row[1]['x1'],hand_row[1]['y2']-20),color,2)
    cv2.imwrite(r'/opt/ice-ai-training-data/page_segment/all_data_v2/mapping/184658pdf_9_bilateral_mapping.png',img_newbox)


In [ ]:
###---->condition1 => hand1(y2)> print(y1)
df_hand1 = df_handcluster[df_handcluster['y2']>py1]
###--->condition2 => print(midy)>hand(y1) or print(midy)>hand(y2)
#(df_hand1['y1']<py_mid) | (df_hand1['y2']>py_mid))
df_hand2 = df_hand1[df_hand1['y1']<py_mid]
#### condition 3 ==>abs(hand(x1)-print(x2))<image.width//2
df_hand3 = df_hand2[abs(df_hand2['x2']-px1)<img.shape[1]//2]
print(df_hand3 )

In [ ]:
img.shape

In [ ]:
np.arange(10,16)

In [ ]:
type(hand_row[1]['y1'])


In [ ]:
np.arange(hand_row[1]['y1'],hand_row[1]['y2']) 

In [ ]:
color_array = np.random.randint(low = 0 , high = 255, size = 3, dtype = int)

In [ ]:
type((color_array.item(0),color_array.item(1),color_array.item(2)))

In [ ]:
type(color.item(0,3))


In [75]:
import tesserocr